# 09 - Diffusion Tensor Imaging (DTI): Fitting and Metrics

This notebook demonstrates how to perform Diffusion Tensor Imaging (DTI) model fitting and calculate common DTI-derived scalar metrics using the `diffusemri` library.

**What is DTI?**
DTI is a widely used model in diffusion MRI that describes water molecule diffusion as a Gaussian process. It models the diffusion characteristics within each voxel using a symmetric 3x3 tensor. From this tensor, various scalar metrics can be derived to quantify aspects of tissue microstructure, such as:
*   **Fractional Anisotropy (FA):** A measure of the degree of anisotropic diffusion (0 for isotropic, 1 for highly anisotropic).
*   **Mean Diffusivity (MD):** The average magnitude of water diffusion.
*   **Axial Diffusivity (AD):** The magnitude of diffusion along the principal diffusion direction.
*   **Radial Diffusivity (RD):** The average magnitude of diffusion perpendicular to the principal direction.

In [ ]:
import os
import shutil
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt

# Dipy imports for gradient table and b-vector generation
from dipy.core.gradients import gradient_table, generate_bvecs 

# diffusemri library imports
from fitting.dti_fitter import fit_dti_volume 

# For conceptual CLI examples:
# from cli.run_dti_fit import main as dti_cli_main 

# Setup a temporary directory for example files
TEMP_DIR = "temp_dti_fitting_example"
if os.path.exists(TEMP_DIR):
    shutil.rmtree(TEMP_DIR)
os.makedirs(TEMP_DIR)

print(f"Temporary directory for examples: {os.path.abspath(TEMP_DIR)}")

# Helper function for plotting slices (can be reused or adapted)
def show_slice(data_vol, slice_idx=None, title="", cmap='gray', vmin=None, vmax=None):
    """Displays a central slice of 3D data or a specific slice of 4D data (first volume)."""
    data_to_show = None
    if data_vol.ndim == 4: # Show first volume of 4D data
        s_idx = slice_idx if slice_idx is not None else data_vol.shape[2] // 2
        data_to_show = data_vol[:, :, s_idx, 0]
    elif data_vol.ndim == 3:
        s_idx = slice_idx if slice_idx is not None else data_vol.shape[2] // 2
        data_to_show = data_vol[:, :, s_idx]
    elif data_vol.ndim == 2: # Already a 2D slice
        data_to_show = data_vol
    else:
        print(f"Cannot display data with {data_vol.ndim} dimensions.")
        return
    
    plt.figure(figsize=(6,5))
    plt.imshow(data_to_show.T, cmap=cmap, origin='lower', vmin=vmin, vmax=vmax)
    plt.title(title)
    plt.xlabel("X voxel index"); plt.ylabel("Y voxel index")
    plt.colorbar(label="Metric Value")
    plt.show()

## Part 1: Preparing Input Data (Synthetic DWI Data)

For a real DTI analysis, you would start with preprocessed DWI data, including the 4D NIfTI image, corresponding b-values and b-vectors files, and ideally a brain mask. 

For this notebook, we will create a small synthetic DWI dataset. This allows us to have predictable characteristics and control the parameters for demonstration purposes.

In [ ]:
# Define data dimensions (small for example)
dims = (20, 20, 5)  # x, y, z dimensions for the image volume
n_volumes = 7      # Number of diffusion volumes (e.g., 1 b0 + 6 diffusion-weighted)

# Create a simple affine matrix (e.g., 2mm isotropic voxels)
affine = np.diag([2.0, 2.0, 2.0, 1.0])

# Define b-values and generate b-vectors
bvals = np.zeros(n_volumes)
bvals[1:] = 1000  # Set b-value of 1000 for diffusion-weighted images, b0 remains 0

bvecs = np.zeros((n_volumes, 3))
# Use Dipy's generate_bvecs for reasonably distributed directions for non-b0 volumes
if n_volumes > 1:
    bvecs[1:] = generate_bvecs(n_volumes - 1) 

# Create a Dipy GradientTable object
gtab = gradient_table(bvals, bvecs, b0_threshold=50)

# Create synthetic DWI data based on the DTI model
dwi_data = np.zeros(dims + (n_volumes,), dtype=np.float32)
S0 = 150.0  # Base signal intensity for b0 images

# Define diffusivities for different regions
d_iso = 0.0010  # Isotropic diffusivity for background (e.g., CSF-like)
d_aniso_parallel = 0.0020  # Higher diffusivity along the principal direction (fiber-like)
d_aniso_perpendicular = 0.0005  # Lower diffusivity perpendicular to the principal direction

# Create an anisotropic region (e.g., a "fiber bundle" along the Y-axis)
x_center, y_center, z_center = dims[0] // 2, dims[1] // 2, dims[2] // 2
fiber_radius_sq = (min(dims[0], dims[2]) / 4)**2 # Radius for the fiber bundle

for i in range(dims[0]):
    for j in range(dims[1]):
        for k in range(dims[2]):
            # Check if the voxel is within the cylindrical fiber region (along Y)
            is_in_fiber = (i - x_center)**2 + (k - z_center)**2 < fiber_radius_sq
            
            for vol_idx in range(n_volumes):
                b_val = gtab.bvals[vol_idx]
                b_vec = gtab.bvecs[vol_idx]
                
                if b_val == 0: # b0 image
                    dwi_data[i, j, k, vol_idx] = S0
                else:
                    if is_in_fiber:
                        # Define diffusion tensor for anisotropic region (principal eigenvector along Y: [0,1,0])
                        # D_aniso = [[d_aniso_perpendicular, 0, 0],
                        #            [0, d_aniso_parallel,    0],
                        #            [0, 0, d_aniso_perpendicular]]
                        # adc = b_vec @ D_aniso @ b_vec.T
                        adc = (b_vec[0]**2 * d_aniso_perpendicular + 
                               b_vec[1]**2 * d_aniso_parallel + 
                               b_vec[2]**2 * d_aniso_perpendicular)
                    else:
                        # Isotropic diffusion for background
                        adc = d_iso 
                    
                    dwi_data[i, j, k, vol_idx] = S0 * np.exp(-b_val * adc)

# Add some Rician noise (simplified as Gaussian for this example, real Rician is more complex)
noise_level = S0 * 0.05 # 5% of S0
dwi_data += np.random.normal(loc=0, scale=noise_level, size=dwi_data.shape)
dwi_data[dwi_data < 0] = 0 # Ensure no negative signals after noise addition
dwi_data = dwi_data.astype(np.float32)

print(f"Generated synthetic DWI data with shape: {dwi_data.shape}")
central_slice_z = dims[2] // 2
show_slice(dwi_data, slice_idx=central_slice_z, title=f"Synthetic DWI (b0, Slice Z={central_slice_z})")
show_slice(dwi_data[..., 1], slice_idx=central_slice_z, title=f"Synthetic DWI (b={bvals[1]}, Slice Z={central_slice_z})")

## Part 2: DTI Model Fitting

Now, we'll fit the DTI model to our synthetic DWI data using the `fit_dti_volume` function from `diffusemri.fitting.dti_fitter`. This function takes the 4D DWI data, b-values, and b-vectors as input and returns a dictionary containing various DTI-derived scalar maps.

In [ ]:
dti_metrics_maps = None
try:
    print("\nFitting DTI model to the synthetic data...")
    # In a real scenario, you would typically provide a brain mask here.
    # For this synthetic data, we'll process all voxels.
    # The b0_threshold from gtab is used implicitly if gtab is used by the fitter, 
    # or explicitly if bvals/bvecs are passed directly.
    dti_metrics_maps = fit_dti_volume(
        image_data_4d=dwi_data,
        b_values=gtab.bvals,    # Pass bvals from GradientTable
        b_vectors=gtab.bvecs,   # Pass bvecs from GradientTable
        b0_threshold=gtab.b0_threshold # Explicitly pass b0_threshold from gtab
        # mask_3d=your_brain_mask_if_you_have_one
    )
    
    print("DTI fitting complete.")
    print(f"Extracted DTI metrics: {list(dti_metrics_maps.keys())}")

    # Visualize some of the key DTI metric maps
    fa_map = dti_metrics_maps.get("FA")
    if fa_map is not None:
        show_slice(fa_map, slice_idx=central_slice_z, title=f"Fractional Anisotropy (FA, Slice Z={central_slice_z})", cmap='viridis', vmin=0, vmax=0.7) # FA typically 0-1
    else:
        print("FA map not found in results.")

    md_map = dti_metrics_maps.get("MD")
    if md_map is not None:
        show_slice(md_map, slice_idx=central_slice_z, title=f"Mean Diffusivity (MD, Slice Z={central_slice_z})", cmap='viridis', vmin=0, vmax=0.003)
    else:
        print("MD map not found in results.")
        
    # You can similarly visualize AD, RD, and the tensor components if needed.
    # ad_map = dti_metrics_maps.get("AD")
    # if ad_map is not None: show_slice(ad_map, slice_idx=central_slice_z, title="Axial Diffusivity (AD)", cmap='viridis', vmin=0, vmax=0.003)
    # rd_map = dti_metrics_maps.get("RD")
    # if rd_map is not None: show_slice(rd_map, slice_idx=central_slice_z, title="Radial Diffusivity (RD)", cmap='viridis', vmin=0, vmax=0.003)

except Exception as e:
    print(f"An error occurred during DTI fitting or visualization: {e}")

## Part 3: Understanding DTI Outputs

The `fit_dti_volume` function returns a dictionary where keys are the names of DTI metrics and values are the corresponding 3D NumPy arrays (maps).

*   **`FA` (Fractional Anisotropy):** Measures the fraction of the tensor's magnitude that can be attributed to anisotropic diffusion. Ranges from 0 (perfectly isotropic diffusion, like in a sphere) to 1 (highly directional diffusion, like along a line). It's sensitive to microstructural integrity and coherence.

*   **`MD` (Mean Diffusivity):** Average diffusivity in all directions. Calculated as the mean of the three eigenvalues of the diffusion tensor. Reflects overall water mobility.

*   **`AD` (Axial Diffusivity):** Diffusivity along the principal axis (the direction of fastest diffusion, corresponding to the largest eigenvalue $\lambda_1$). Often associated with axonal integrity.

*   **`RD` (Radial Diffusivity):** Average diffusivity in the directions perpendicular to the principal axis. Calculated as the mean of the two smaller eigenvalues ($\frac{\lambda_2 + \lambda_3}{2}$). Often associated with myelin integrity.

*   **`D_tensor_map`:** A 5D NumPy array (`X, Y, Z, 3, 3`) containing the fitted 3x3 symmetric diffusion tensor for each voxel. From this tensor, all other DTI metrics are derived.

The synthetic data created an anisotropic region along the Y-axis. You should observe higher FA values in this region compared to the isotropic background. MD might be similar or slightly different based on the chosen diffusivities. AD should be high along Y in the fiber region, and RD should be lower.

## Part 4: CLI Usage (Conceptual)

DTI fitting can also be performed using the `run_dti_fit.py` command-line script provided with `diffusemri`.

```bash
# Conceptual CLI command - replace paths with your actual file paths
# First, save your dwi_data, bvals, and bvecs to files if they are in memory.
# For example:
# nib.save(nib.Nifti1Image(dwi_data, affine), os.path.join(TEMP_DIR, 'synthetic_dwi.nii.gz'))
# np.savetxt(os.path.join(TEMP_DIR, 'synthetic.bval'), bvals, fmt='%d')
# np.savetxt(os.path.join(TEMP_DIR, 'synthetic.bvec'), bvecs.T, fmt='%.8f') # Note transpose for FSL format

python cli/run_dti_fit.py \
  --dwi_file /path/to/your/dwi.nii.gz \
  --bval_file /path/to/your/bvals.bval \
  --bvec_file /path/to/your/bvecs.bvec \
  --mask_file /path/to/your/brain_mask.nii.gz \  # Optional, but highly recommended for real data
  --output_prefix /path/to/your_output_dir/dti_results_cli 
  # This will generate files like dti_results_cli_FA.nii.gz, dti_results_cli_MD.nii.gz, etc.
```
*(To run this directly in the notebook, you would typically use `!python ...` and ensure paths are correct, or use the `dti_cli_main` Python function if it's designed to be called with arguments.)*

## Cleanup

Remove the temporary directory and its contents.

In [ ]:
if os.path.exists(TEMP_DIR):
    shutil.rmtree(TEMP_DIR)
    print(f"Cleaned up temporary directory: {TEMP_DIR}")